# Downloading the selected datasets from NASA Earthdata

In [ ]:
!pip install cartopy earthaccess netCDF4 pydap

In [ ]:
import os
import pprint
import re
import tempfile
import warnings
from base64 import b64encode
from datetime import datetime
from getpass import getpass
from typing import List, Tuple, Any
from urllib.parse import quote

import earthaccess
import google.colab
import h5py
import numpy as np
import requests
import xarray as xr
from pydap.client import open_url

pp = pprint.PrettyPrinter(indent=4)

## EDL token

The easiest way to get authentication is to obtain an Earthdata Login token (EDL token) and use it for requests. One way of obtaining the EDL token is described in the section 3 of [this](https://github.com/nasa/gesdisc-tutorials/blob/main/notebooks/How_to_Generate_Earthdata_Prerequisite_Files.ipynb) jupyter notebook.

You can call the function `save_edl_token_to_file()` below if you need to. You will have to specify the NASA Earthdata account login and password, so complete the registration procedure beforehand. The detailed manual can be found [here](https://disc.gsfc.nasa.gov/information/documents?title=Data%20Access).

In [ ]:
def save_edl_token_to_file(token_file_path: str = "edl_token.txt"):
    # Earthdata Login URL for obtaining the token, and creating one if it doesn't exist
    url = 'https://urs.earthdata.nasa.gov/api/users/find_or_create_token'

    # Earthdata Login credential prompts
    prompts = {
        'username': 'Enter NASA Earthdata Login Username \n(or create an account at urs.earthdata.nasa.gov): ',
        'password': 'Enter NASA Earthdata Login Password: '
    }

    # Get credentials from user input
    username = getpass(prompt=prompts['username'])
    password = getpass(prompt=prompts['password'])

    # Encode credentials using Base64
    credentials = b64encode(f"{username}:{password}".encode('utf-8')).decode('utf-8')

    # Headers with the Basic Authorization
    headers = {
        'Authorization': f'Basic {credentials}'
    }

    # Make the POST request to get the token
    response = requests.post(url, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the response JSON to get the token
        token_info = response.json()
        token = token_info.get("access_token")
        print("Token retrieved successfully")

        # Write the token to the .edl_token file
        with open(token_file_path, 'w') as token_file:
            token_file.write(token)

        print(f"Token saved to {token_file_path}")

    else:
        print("Failed to retrieve token:", response.text)


## Call `save_edl_token_to_file()` below if you need to

In [ ]:
if False:
    save_edl_token_to_file()

If the token was saved to `edl_token.txt`, you can see the contents of the file by running `!cat edl_token.txt` in a code cell or by double-clicking on the file name in the file explorer.

## Paste your EDL token (and check authentication)

In [ ]:
if 'EARTHDATA_TOKEN' not in os.environ:
    # paste your token securely (it won't show on-screen)
    os.environ['EARTHDATA_TOKEN'] = getpass('Paste your EDL token: ')

# initialize an authenticated session
session = earthaccess.login(strategy='environment')   # uses EARTHDATA_TOKEN
print('Authenticated?', session.authenticated)
print()
print(f"{session=}")
print(f"{session.get_session()=}")

## Set the dates and region of interest

In [ ]:
#----------------------------------------------------------------
# (!!!) SET THESE VARS MANUALLY:

#Ladoga:
LAT_MIN = 59.5
LAT_MAX = 62.6
LON_MIN = 28.0
LON_MAX = 33.8

DATE_MIN = '2025-11-01'
DATE_MAX = '2025-11-02'

PRODUCT = 'FS'
OBSERVABLE_VARS = [
    f'/{PRODUCT}/VER/sigmaZeroNPCorrected',
]
OBSERVABLE_NAME_TO_COLORBAR_TITLE = {
    f'/{PRODUCT}/VER/sigmaZeroNPCorrected': "NRCS Ku-band (dB)"
}
SCAN_TIME_REQUIRED = False

HDF5_BNAME_DELIMITER = '.'
HDF5_BNAME_TRACK_NUMBER_PART_IDX = -2
# ^ (-2) for 'GPM_2ADPR.07:2A.GPM.DPR.V9-20240130.20251101-S023933-E041248.066262.V07C.HDF5'
HDF5_BNAME_TRACK_START_PART_IDX = -3
# ^ (-3) for 'GPM_2ADPR.07:2A.GPM.DPR.V9-20240130.20251101-S023933-E041248.066262.V07C.HDF5'
#----------------------------------------------------------------

# These vars are inferred from the vars defined above
requested_vars_slashes = [
    f'/{PRODUCT}/Latitude',
    f'/{PRODUCT}/Longitude',
]
if SCAN_TIME_REQUIRED:
    requested_vars_slashes.extend([
        f'/{PRODUCT}/ScanTime/Year',
        f'/{PRODUCT}/ScanTime/Month',
        f'/{PRODUCT}/ScanTime/DayOfMonth',
        f'/{PRODUCT}/ScanTime/DayOfYear',
        f'/{PRODUCT}/ScanTime/Hour',
        f'/{PRODUCT}/ScanTime/Minute',
        f'/{PRODUCT}/ScanTime/Second',
        f'/{PRODUCT}/ScanTime/MilliSecond',
    ])

requested_vars_slashes.extend(OBSERVABLE_VARS)

requested_vars_underscores = [v[1:].replace('/', '_') for v in requested_vars_slashes]

## Utility functions (data search)

Feel free to fold the code of each function if you are not going to modify the code.

In [ ]:
def extract_track_number_from_h5_url_or_fpath(h5_url_or_fpath: str) -> str:
    fname = h5_url_or_fpath.split("/")[-1]
    bname = os.path.splitext(fname)[0]
    parts = bname.split(HDF5_BNAME_DELIMITER)
    track_number = parts[HDF5_BNAME_TRACK_NUMBER_PART_IDX]
    return track_number

def extract_track_start_timestamp_from_h5_url_or_fpath(h5_url_or_fpath: str) -> str:
    """
    Extract the track start timestamp from an HDF5 filename or URL and return a datetime object.

    Expected part format: "<yyyy><mm><dd>-S<hh><mm><ss>-E<hh><mm><ss>"
    Example part: "20251101-S023933-E041248" -> returns datetime(2025,11,1,2,39,33)

    Raises ValueError if the expected pattern cannot be found.
    """
    fname = h5_url_or_fpath.split("/")[-1]
    bname = os.path.splitext(fname)[0]
    parts = bname.split(HDF5_BNAME_DELIMITER)
    track_timestamps = parts[HDF5_BNAME_TRACK_START_PART_IDX]

    m = re.search(r'(?P<date>\d{8})-S(?P<start>\d{6})', track_timestamps)
    if not m:
        raise ValueError(f"Couldn't parse start timestamp from '{track_timestamps}' in '{h5_url_or_fpath}'")

    date = m.group('date')   # YYYYMMDD
    start = m.group('start') # HHMMSS

    year = int(date[0:4])
    month = int(date[4:6])
    day = int(date[6:8])

    hour = int(start[0:2])
    minute = int(start[2:4])
    second = int(start[4:6])

    return datetime(year, month, day, hour, minute, second)


def get_search_data_results() -> list[str]:
    temporal = (
        f"{DATE_MIN}T00:00:00Z",
        f"{DATE_MAX}T23:59:59Z"
    )

    results = earthaccess.search_data(
        short_name="GPM_2ADPR",
        bounding_box=(LON_MIN, LAT_MIN, LON_MAX, LAT_MAX),
        temporal=temporal,
        count=-1,
    )

    return results


def find_opendap_url(granule_meta: dict) -> str | None:
    """Return the first OPeNDAP URL found in the granule UMM RelatedUrls (or None)."""
    related = granule_meta.get("umm", {}).get("RelatedUrls", []) or []
    for r in related:
        url = r.get("URL") or r.get("url") or r.get("Href") or r.get("Href") if isinstance(r, dict) else None
        if not url:
            continue
        if "opendap" in url.lower():
            return url
        # sometimes Type field holds the service type
        if "Type" in r and (("opendap" in (r.get("Type") or "").lower()) or ("OPeNDAP" in (r.get("Type") or ""))):
            return url

    # fallback: sometimes earthaccess.data_links() returns https download links only
    # but many collections include OPENDAP in 'RelatedUrls' as above
    return None


def read_latlon_from_localfile(fobj) -> Tuple[np.ndarray, np.ndarray, List[str]]:
    """
    Given a file-like object or path to a GPM HDF5 granule (opened by earthaccess.open),
    save locally if necessary and use h5py to find latitude/longitude datasets.
    Returns lat, lon (numpy arrays) and list of candidate data dataset paths found in file.
    """
    # Ensure we have a filesystem path for h5py
    if hasattr(fobj, "name") and isinstance(fobj.name, str):
        local_path = fobj.name
    else:
        tmpf = tempfile.NamedTemporaryFile(suffix=".HDF5", delete=False)
        tmp_path = tmpf.name
        tmpf.write(fobj.read())
        tmpf.close()
        local_path = tmp_path

    # open file with h5py
    lat = lon = None
    candidate_data_paths = []

    with h5py.File(local_path, "r") as hf:
        # walk file looking for dataset names that include 'lat' / 'lon' / 'latitude' / 'longitude'
        def visitor(name, obj):
            nonlocal lat, lon, candidate_data_paths
            if isinstance(obj, h5py.Dataset):
                lower = name.lower()
                if ("latitude" in lower) or ("lat" in lower and "quality" not in lower):
                    # pick first reasonable-shaped lat dataset
                    try:
                        arr = obj[()]
                    except Exception:
                        arr = None
                    if arr is not None and lat is None and arr.size > 0:
                        lat = np.array(arr)
                if ("longitude" in lower) or ("lon" in lower and "quality" not in lower):
                    try:
                        arr = obj[()]
                    except Exception:
                        arr = None
                    if arr is not None and lon is None and arr.size > 0:
                        lon = np.array(arr)
                # heuristically collect candidate data variable paths (2D arrays)
                if obj.ndim >= 2 and obj.size > 0 and "latitude" not in lower and "longitude" not in lower:
                    candidate_data_paths.append(name)
        hf.visititems(visitor)

    return lat, lon, candidate_data_paths


def indices_intersecting_bbox(
    lat: np.ndarray,
    lon: np.ndarray,
    latmin: float,
    latmax: float,
    lonmin: float,
    lonmax: float,
) -> np.ndarray:
    """
    Given lat and lon arrays (typically 2D: scans x pixels), return an array of scan indices
    (0-based) that have at least one pixel inside the bbox.
    """
    if lat is None or lon is None:
        return np.array([], dtype=int)

    # normalize shapes: if 1D vs 2D
    if lat.ndim == 1 and lon.ndim == 1:
        # 1D along scans
        mask = (lat >= latmin) & (lat <= latmax) & (lon >= lonmin) & (lon <= lonmax)
        return np.where(mask)[0]

    if lat.ndim == 2 and lon.ndim == 2:
        # assume axis 0 = scans, axis 1 = pixels OR vice versa; find which axis is longer (scans)
        # better: treat axis 0 as scans if first axis length > second (typical)
        if lat.shape[0] >= lat.shape[1]:
            scan_axis = 0
            pixel_axis = 1
        else:
            scan_axis = 1
            pixel_axis = 0
            # transpose to make scans axis 0
            lat = lat.T
            lon = lon.T

        mask = (lat >= latmin) & (lat <= latmax) & (lon >= lonmin) & (lon <= lonmax)
        # check any pixel in each scan
        scan_mask = np.any(mask, axis=1)
        return np.where(scan_mask)[0]

    # fallback: try flatten and match approximate
    mask = (lat >= latmin) & (lat <= latmax) & (lon >= lonmin) & (lon <= lonmax)
    if mask.any():
        return np.array([0])  # we can't identify scans reliably, return the file-level match

    return np.array([], dtype=int)


def contiguous_ranges(idxs: np.ndarray) -> List[Tuple[int,int]]:
    """Turn sorted 1D idx array into list of (start, end) inclusive ranges."""
    if idxs.size == 0:
        return []
    idxs = np.sort(idxs)
    ranges = []
    start = prev = idxs[0]
    for i in idxs[1:]:
        if i == prev + 1:
            prev = i
            continue
        else:
            ranges.append((start, prev))
            start = prev = i
    ranges.append((start, prev))
    return ranges


## Download the sliced arrays

In [ ]:
results = get_search_data_results()

track_number_and_start_time = {}
track_number_to_arr_dict = {}

# main loop: for each granule found by your search
for g in results:
    title = g.get("meta", {}).get("title") or g.get("meta", {}).get("granule_ur") or g.get("umm", {}).get("GranuleUR")
    print("\n=== Granule:", title)
    opendap_base = find_opendap_url(g)
    if not opendap_base:
        print(" No OPeNDAP URL found in RelatedUrls for this granule; skipping.")
        continue
    print(" OPeNDAP URL (base):", opendap_base)

    fileobjs = earthaccess.open([g])
    fobj = fileobjs[0]
    # open the granule once with earthaccess (handles auth for you)
    try:
        # read lat/lon and candidate data paths
        lat, lon, candidate_data_paths = read_latlon_from_localfile(fobj)
    except Exception as e:
        warnings.warn(f"Failed to open granule with earthaccess.open(): {e}")
        lat = lon = None
        candidate_data_paths = []

    if lat is None or lon is None:
        print(" Could not locate lat/lon arrays inside file. Candidate data paths (heuristic):")
        for p in candidate_data_paths[:10]:
            print("  -", p)
        print(" You may need to inspect file structure manually. Skipping.")
        continue

    # find scan indices that intersect your box
    idxs = indices_intersecting_bbox(lat, lon, LAT_MIN, LAT_MAX, LON_MIN, LON_MAX)
    print(" Scan indices intersecting ROI (count):", idxs.size)
    if idxs.size == 0:
        print(" No scans in this granule intersect the bounding box.")
        continue

    ranges = contiguous_ranges(idxs)
    #print(" Contiguous scan ranges (inclusive):", ranges)
    single_enclosing_range = (
        min(left for left, right in ranges),
        max(right for left, right in ranges)
    )
    print(" Scan range (inclusive):", single_enclosing_range)

    # compute pixel range (use full pixel range)
    if lat.ndim == 2:
        # ensure we computed scans as axis 0 above
        n_pixels = lat.shape[1] if lat.shape[0] >= lat.shape[1] else lat.shape[0]
    else:
        n_pixels = 0

    # pick variables to subset: use requested_vars_slashes if provided, otherwise pick top 2 candidate_data_paths
    if requested_vars_slashes:
        vars_to_request = requested_vars_slashes
    else:
        # prefer candidate paths that are clearly data (not quality flags)
        filtered = [p for p in candidate_data_paths if all(x not in p.lower() for x in ("qual","status","flag","count"))]
        if not filtered:
            filtered = candidate_data_paths
        # take first 3 candidates to request by default
        vars_to_request = filtered[:3]

    print(" Will request variables:", vars_to_request)

    # For each contiguous range, build constraint expression and attempt to open via xarray+pydap
    sidx, eidx = [int(idx) for idx in single_enclosing_range]

    subset_url_dap4 = opendap_base.replace("https://", "dap4://")
    print("subset_url_dap4 (trying):", subset_url_dap4)

    try:
        ds_pydap = open_url(subset_url_dap4, session=session.get_session())
        print(" Successfully called `open_url(subset_url_dap4, session=session.get_session())`.")
        print(" Variables:", list(ds_pydap.keys()))
    except Exception as e:
        warnings.warn(f"xarray/pydap failed to open subset_url: {e}")
        print(" You can either: (1) inspect the exact variable names in the OPeNDAP DDS/DAS,")
        print(" or (2) download the granule locally and slice after loading with h5py/xarray.")
        # continue to next range
        continue

    selected_np_arrays = {}
    for var_name_slashes in requested_vars_slashes:
        var_name_underscores = var_name_slashes[1:].replace('/', '_')
        arr_obj = (
            ds_pydap[var_name_underscores][sidx:eidx]
            if n_pixels > 0
            else ds_pydap[var_name_underscores]
        )
        selected_np_arrays[var_name_slashes] = np.array(arr_obj)

    track_number = extract_track_number_from_h5_url_or_fpath(title)
    track_number_to_arr_dict[track_number] = selected_np_arrays

    track_number_and_start_time[title] = (
        track_number,
        extract_track_start_timestamp_from_h5_url_or_fpath(title).isoformat().replace("T", " ")
    )



In [ ]:
print("Granule name -> (track_number, track_start_datetime)")
pp.pprint(track_number_and_start_time)

print()
print("Structure of the obtained data (track numbers, variable names, numpy array dtypes and shapes):")
pp.pprint(
    {
        track_number: {k: (v.dtype, v.shape) for k, v in arr_dict.items()}
        for track_number, arr_dict in track_number_to_arr_dict.items()
    }
)

## Saving the arrays

In [ ]:
dict_to_save = {
    granule_name: track_number_to_arr_dict[track_number]
    for granule_name, (track_number, track_start_time) in track_number_and_start_time.items()
}

if False:
    pp.pprint(
        {
            granule_name: {k: (v.dtype, v.shape) for k, v in arr_dict.items()}
            for granule_name, arr_dict in dict_to_save.items()
        }
    )

np.savez_compressed(
    f"./from_{DATE_MIN}_to_{DATE_MAX}",
    **dict_to_save,
)

In [ ]:
!ls -la

## Visualizing the arrays

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, LabelSet, Range1d, ColorBar, HoverTool, LinearColorMapper
from bokeh.palettes import Turbo256
from bokeh.transform import transform

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from shapely.geometry import LineString, MultiLineString, MultiPolygon, Polygon

output_notebook()

In [ ]:
#----------------------------------------------------------------
# (!!!) SET THESE VARS MANUALLY:

SINGLE_SELECTED_OBSERVABLE_VAR = f'/{PRODUCT}/VER/sigmaZeroNPCorrected'
SLICE_FOR_SELECTED_OBSERVABLE_VAR = (slice(None), slice(None), 0)
# ^ (slice(None), slice(None), 0) corresponds to the [:, :, 0] slicing

SHOW_RIVERS = True

ADD_HOVER_TOOL = False
BOKEH_FIGURE_MAX_SIZE = 800
BOKEH_FIGURE_MARKER_SIZE = 6
#----------------------------------------------------------------


## Utility functions (data visualization using bokeh)

In [ ]:
def prepare_bokeh_figure(
    plot_title: str,
) -> figure:
    # Extract bounding box coordinates
    lon_min, lon_max = LON_MIN, LON_MAX
    lat_min, lat_max = LAT_MIN, LAT_MAX

    try:
        height_to_width_ratio = (lat_max - lat_min) / (lon_max - lon_min)
    except ZeroDivisionError:
        height_to_width_ratio = 1.0

    if height_to_width_ratio > 1.0:
        fig_height = BOKEH_FIGURE_MAX_SIZE
        fig_width = int(fig_height / height_to_width_ratio)
    else:
        fig_width = BOKEH_FIGURE_MAX_SIZE
        fig_height = int(fig_width * height_to_width_ratio)

    # Create Bokeh figure
    p = figure(
        title=plot_title,
        x_range=Range1d(lon_min, lon_max, bounds=(lon_min, lon_max)),
        y_range=Range1d(lat_min, lat_max, bounds=(lat_min, lat_max)),
        width=fig_width,
        height=fig_height,
        tools="pan,wheel_zoom,box_zoom,reset,save",
        toolbar_location="left",
    )

    # Configure plot appearance
    p.title.text_font_size = "16pt"

    return p


def get_geojson_source(feature):
    """Convert cartopy feature to Bokeh ColumnDataSource"""
    xs, ys = [], []

    for geom in feature.geometries():
        if geom.is_empty:
            continue

        # Normalize Multi* into parts
        parts = getattr(geom, "geoms", [geom])
        for g in parts:
            gt = g.geom_type

            if gt in ("LineString", "LinearRing"):
                x, y = g.xy
                xs.append(list(x))
                ys.append(list(y))

            elif gt == "MultiLineString":
                for line in g.geoms:
                    x, y = line.xy
                    xs.append(list(x))
                    ys.append(list(y))

            elif gt == "Polygon":
                # outline (exterior)
                x, y = g.exterior.xy
                xs.append(list(x))
                ys.append(list(y))
                # (optional) holes
                # for ring in g.interiors:
                #     rx, ry = ring.xy
                #     xs.append(list(rx))
                #     ys.append(list(ry))

            elif gt == "MultiPolygon":
                for poly in g.geoms:
                    x, y = poly.exterior.xy
                    xs.append(list(x))
                    ys.append(list(y))
                    # (optional) holes
                    # for ring in poly.interiors:
                    #     rx, ry = ring.xy
                    #     xs.append(list(rx))
                    #     ys.append(list(ry))

    return ColumnDataSource(dict(xs=xs, ys=ys))


def get_land_polygons():
    """Extract land polygons from Natural Earth features"""
    land_geoms = cfeature.NaturalEarthFeature("physical", "land", "50m")
    polygons = []

    for geom in land_geoms.geometries():
        if isinstance(geom, Polygon):
            polygons.append(geom)
        elif isinstance(geom, MultiPolygon):
            polygons.extend(geom.geoms)

    return polygons


def get_polygon_source(polygons, projection):
    """Convert polygons to Bokeh ColumnDataSource"""
    xs, ys = [], []

    for polygon in polygons:
        # Extract exterior coordinates
        x, y = polygon.exterior.xy
        # Project coordinates if needed
        if projection:
            x, y = projection.transform_points(
                ccrs.PlateCarree(), np.array(x), np.array(y)
            )[:, :2].T
        xs.append(x.tolist())
        ys.append(y.tolist())

    return ColumnDataSource(data=dict(xs=xs, ys=ys))


def draw_earth_features(
    p: figure,
) -> figure:
    water_related_geoms = cfeature.NaturalEarthFeature("physical", "coastline", "50m")
    water_related_source = get_geojson_source(water_related_geoms)
    p.multi_line(
        xs="xs", ys="ys",
        source=water_related_source,
        line_color="black",
        line_width=1,
    )

    water_related_geoms = cfeature.NaturalEarthFeature("physical", "lakes", "10m")
    water_related_source = get_geojson_source(water_related_geoms)
    p.patches(
        xs="xs", ys="ys",
        source=water_related_source,
        fill_alpha=0.0,
        line_color="black",
        line_width=1,
    )

    if SHOW_RIVERS:
        water_related_geoms = cfeature.NaturalEarthFeature("physical", "rivers_lake_centerlines", "10m")
        water_related_source = get_geojson_source(water_related_geoms)
        p.multi_line(
            xs="xs", ys="ys",
            source=water_related_source,
            line_color="blue",
            line_width=1.5,
            line_cap="round",
            line_join="round",
        )

    land_polygons = get_land_polygons()
    land_source = get_polygon_source(land_polygons, projection=None)
    p.patches(
        xs="xs",
        ys="ys",
        source=land_source,
        fill_color="#E0E0E0",  # Light gray
        fill_alpha=0.0,
        line_color="black",  # Outline color
        line_width=0.5,  # Outline thickness
    )

    return p


def draw_points_colorbar(
    p: Any,
    source: ColumnDataSource,
    observable: np.ndarray,
):
    color_mapper = LinearColorMapper(
        palette=Turbo256,
        low=min(observable),
        high=max(observable),
    )

    p.scatter(
        "longitude",
        "latitude",
        source=source,
        marker="circle",
        size=BOKEH_FIGURE_MARKER_SIZE,
        fill_color=transform("observable", color_mapper),
        fill_alpha=1.0, #0.5,
        line_color=None,
    )

    observable_print_name = OBSERVABLE_NAME_TO_COLORBAR_TITLE[OBSERVABLE_VARS[0]]

    if ADD_HOVER_TOOL:
        tooltips = [
            ("(lat, lon)", "(@latitude{0.000}, @longitude{0.000})"),
            (observable_print_name, "@observable{0.00}"),
        ]
        hover = HoverTool(tooltips=tooltips)
        p.add_tools(hover)

    color_bar = ColorBar(
        color_mapper=color_mapper,
        label_standoff=12,
        width=8,
        location=(0, 0),
        title=observable_print_name,
        title_text_font_size="16pt",
        title_text_font_style="normal",
        major_label_text_font_size="14pt",
    )
    p.add_layout(color_bar, "right")


def visualize_single_track(
    track_order_number: int,
    track_number: str,
    track_number_to_arr_dict: dict[str, str],
    track_number_and_start_time: dict[str, tuple[str, str]],
):
    track_number_to_start_timestamp = {
        track_number: start_timestamp
        for granule_name, (track_number, start_timestamp) in track_number_and_start_time.items()
    }
    p = prepare_bokeh_figure(
        f"({track_order_number}) Track number {track_number}"
        f"\nTrack started {track_number_to_start_timestamp[track_number]}",
    )

    arr_dict = track_number_to_arr_dict[track_number]
    latitude_key = [var_name for var_name in requested_vars_slashes if 'latitude' in var_name.lower()][0]
    latitude = arr_dict[latitude_key]
    longitude_key = [var_name for var_name in requested_vars_slashes if 'longitude' in var_name.lower()][0]
    longitude = arr_dict[longitude_key]

    observable = arr_dict[SINGLE_SELECTED_OBSERVABLE_VAR][SLICE_FOR_SELECTED_OBSERVABLE_VAR].flatten()

    source = ColumnDataSource(
        data=dict(
            latitude=latitude.flatten(),
            longitude=longitude.flatten(),
            observable=observable,
        )
    )

    draw_points_colorbar(
        p,
        source,
        observable,
    )

    p = draw_earth_features(p)

    show(p)


In [ ]:
#google.colab.output.no_vertical_scroll()

for track_order_number, (track_number, _) in enumerate(track_number_to_arr_dict.items(), start=1):
    visualize_single_track(
        track_order_number,
        track_number,
        track_number_to_arr_dict,
        track_number_and_start_time,
    )

# It is probably a good idea to click
# 'View output fullscreen' in the 'Code cell output actions' menu